In [87]:
import numpy as np
from sympy import symbols, diff, sin

def gradient_descent(f, lr, x_t, max_iter, tol):
    #h = 1e-5  # Small step for numerical differentiation
    #y_diff = (f(x0 + h) - f(x0)) / h
    x = symbols('x')
    f_sym = f(x)
    y_diff = diff(f_sym, x)
    i = 0
    while i <= max_iter:
        grad = float(y_diff.subs(x, x_t))
        x_t1 = x_t - lr * grad
        if np.abs(grad) < tol:
            break
        x_t = x_t1
        i += 1

    return f_sym.subs(x, x_t), grad

gradient_descent(
    f=lambda x: x**2 + 5*sin(x),
    lr=0.1,
    x_t=-10,
    max_iter=100,
    tol=1e-1
)

(-3.24623879507347, -0.04492266828968239)

In [88]:
from sympy import symbols, diff, Matrix

def gradient(f, vars):
    grad = [diff(f, var) for var in vars]
    return Matrix(grad)


In [89]:
def gradient_descent_variables(f, lr, x_t, max_iter, tol):
    # define vector gradient for multiple variables
    x, y = symbols('x y')  # Define symbols for x and y
    grad = gradient(f, [x, y])
    print(grad)  # Use f_sym here
    i = 0
    while i <= max_iter:
        grad_values = [float(grad[0].subs({x: x_t[0], y: x_t[1]})), float(grad[1].subs({x: x_t[0], y: x_t[1]}))]
        x_t1 = [x_t[0] - lr * grad_values[0], x_t[1] - lr * grad_values[1]]
        if np.linalg.norm(grad_values) < tol:
            break
        x_t = x_t1
        i += 1

    return f.subs({x: x_t[0], y: x_t[1]}), x_t[0], x_t[1]

x, y = symbols('x y')  # Define symbols for x and y globally

gradient_descent_variables(
    f=(x**2 + y - 7)**2 + (x - y + 1)**2,
    lr=0.015,
    x_t=[0, 0],
    max_iter=1000,
    tol=1e-3
)

Matrix([[4*x*(x**2 + y - 7) + 2*x - 2*y + 2], [2*x**2 - 2*x + 4*y - 16]])


(3.83185466235675e-7, -2.999855012457979, -1.9992471632382347)

In [102]:
import numpy as np

def gradient(f, x, eps=1e-8):
    grad = np.zeros_like(x)
    fx = f(x)
    for i in range(len(x)):
        dx = np.zeros_like(x)
        dx[i] = eps
        grad[i] = (f(x + dx) - fx) / eps
    return grad

def accelerated_gradient_descent(f, x0, lr, N, tol):
    x_prev = np.copy(x0)
    x = np.copy(x0)
    i = 0

    while i <= N - 1:
        y = x + (i - 1) / (i + 2) * (x - x_prev)      
        grad_y = gradient(f, y)
        x_next = y - lr * grad_y

        if np.linalg.norm(gradient(f, x_next)) < tol:
            return x_next

        x_prev = x
        x = x_next
        i += 1

    return x

f = lambda x: (x[0]**2 + x[1] - 7)**2 + (x[0] - x[1] + 1)**2

x0 = np.array([0, 0])
x_min, success = accelerated_gradient_descent(f, x0, lr=0.015, N=1000, tol=1e-3)

print("Nghiệm gần đúng:", x_min)


Nghiệm gần đúng: -2.9998812889419706


In [ ]:
def Accelerated_gradient_descent(f, lr, x_t, max_iter, tol):
    # define vector gradient for multiple variables
    x, y = symbols('x y')
    grad = gradient(f, [x, y])
    i = 0
    x_t1 = x_t  # Initialize x_t1 with the initial value of x_t
    while i <= max_iter - 1:
        f = x_t + (i - 1)/(i + 2) * (np.array(x_t) - np.array(x_t1))
        grad = [float(grad.subs({x: x_t[0], y: x_t[1]})), float(grad.subs({x: x_t[0], y: x_t[1]}))]
        x_t2 = [f[0] - lr * grad[0], f[1] - lr * grad[1]]
        if np.linalg.norm(grad) < tol:
            break
        x_t = x_t1
        x_t1 = x_t2
        i += 1

    return f.subs({x: x_t[0], y: x_t[1]}), x_t[0], x_t[1]

Accelerated_gradient_descent(
    f=lambda x, y: (x**2 + y - 7)**2 + (x - y + 1)**2,
    lr=0.015,
    x_t=[0, 0],
    max_iter=1000,
    tol=1e-3
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_448\826700701.py:12: RuntimeWarning: invalid value encountered in scalar subtract
  x_t2 = [f[0] - lr * grad[0], f[1] - lr * grad[1]]


(nan, np.float64(nan), np.float64(nan))

In [33]:
import numpy as np
import pandas as pd

df = pd.read_csv('tuoi_hatt.csv')
print(df.head())
# Chuyển dữ liệu từ DataFrame về numpy arrays
# Inspect column names to ensure correct usage
print(df.columns)

# Use the correct column names
a = df['Tuổi'].values.reshape(-1, 1)  # đặc trưng (feature)
b = df['HATT'].values  # nhãn (label)

# Thêm bias term vào a (thành [x, 1] để học cả hệ số chặn)
a_bias = np.hstack([a, np.ones_like(a)])

# Tạo danh sách loss function f_i và gradient tương ứng cho SGD
f_list = []
grad_list = []

for i in range(len(a)):
    ai = a_bias[i]
    bi = b[i]
    f_i = lambda x, ai=ai, bi=bi: 0.5 * (np.dot(ai, x) - bi)**2
    grad_i = lambda x, ai=ai, bi=bi: ai * (np.dot(ai, x) - bi)
    f_list.append(f_i)
    grad_list.append(grad_i)

# Hàm SGD đã định nghĩa trước đó
def stochastic_gradient_descent(f_list, grad_list, x0, eta, N, epsilon=1e-6):
    m = len(f_list)
    x = np.copy(x0)
    
    for i in range(N):
        idx = np.random.randint(0, m)
        grad = grad_list[idx](x)
        x_new = x - eta * grad

        full_grad = np.sum([g(x_new) for g in grad_list], axis=0) / m
        if np.linalg.norm(full_grad) < epsilon:
            return x_new, True

        x = x_new

    return x, False

# Gọi hàm SGD với vector khởi tạo x0 = [0, 0]
x0 = np.zeros(a_bias.shape[1])
x_result, success = stochastic_gradient_descent(f_list, grad_list, x0, eta=0.001, N=5000)

x_result, success


   STT  Tuổi  HATT
0    1     39   144
1    2     36   136
2    3     45   138
3    4     47   145
4    5     65   162
Index(['STT', 'Tuổi', 'HATT'], dtype='object')


(array([-2.46944233e+241,  2.75781515e+240]), False)

In [ ]:
# Dự đoán tuổi khi biết huyết áp tâm thu
X = df["HATT"].values.reshape(-1, 1)
y = df['Tuổi'].values

X_bias = np.hstack([X, np.ones_like(X)])

f_list_age = []
grad_list_age = []

for i in range(len(X_bias)):
    xi = X_bias[i]
    yi = y[i]
    f_i = lambda w, xi=xi, yi=yi: 0.5 * (np.dot(xi, w) - yi)**2
    grad_i = lambda w, xi=xi, yi=yi: xi * (np.dot(xi, w) - yi)
    f_list_age.append(f_i)
    grad_list_age.append(grad_i)

w0 = np.zeros(X_bias.shape[1])
w_result, success_age = stochastic_gradient_descent(f_list_age, grad_list_age, w0, eta=0.00001, N=5000)
HATT = 150
age = w_result[0]*HATT-w_result[1]
print(age)

44.70210023313253


In [72]:
# Đảo lại: input là Tuổi, output là HATT
X2 = df["Tuổi"].values.reshape(-1, 1)  # input: Tuổi
y2 = df["HATT"].values  # output: HATT

# Thêm bias vào input
X2_bias = np.hstack([X2, np.ones_like(X2)])

# Tạo danh sách hàm mất mát và gradient
f_list_bp = []
grad_list_bp = []

for i in range(len(X2_bias)):
    xi = X2_bias[i]
    yi = y2[i]
    f_i = lambda w, xi=xi, yi=yi: 0.5 * (np.dot(xi, w) - yi)**2
    grad_i = lambda w, xi=xi, yi=yi: xi * (np.dot(xi, w) - yi)
    f_list_bp.append(f_i)
    grad_list_bp.append(grad_i)

# Vector trọng số khởi tạo
w0_bp = np.zeros(X2_bias.shape[1])

# Chạy SGD để huấn luyện mô hình dự đoán HATT từ tuổi
w_result_bp, success_bp = stochastic_gradient_descent(f_list_bp, grad_list_bp, w0_bp, eta=0.00001, N=5000)

w_result_bp, success_bp
tuoi_test = 50
hatt_du_doan = w_result_bp[0] * tuoi_test + w_result_bp[1]
print(f"Dự đoán HATT cho người {tuoi_test} tuổi:", hatt_du_doan)

Dự đoán HATT cho người 50 tuổi: 141.80691245488586


In [ ]:
def Stochastic_gradient_descent(a, b, x_t, max_iter, tol):
    # define vector gradient for multiple variables
    x, y = symbols('x y')
    f_sym = f(x, y)
    grad_x = diff(f_sym, x)
    grad_y = diff(f_sym, y)
    i = 0
    while i <= max_iter - 1:
        grad = [float(grad_x.subs({x: x_t[0], y: x_t[1]})), float(grad_y.subs({x: x_t[0], y: x_t[1]}))]
        x_t1 = [x_t[0] - lr * grad[0], x_t[1] - lr * grad[1]]
        if np.linalg.norm(grad) < tol:
            break
        x_t = x_t1
        i += 1

    return f_sym.subs({x: x_t[0], y: x_t[1]}), x_t[0], x_t[1]

Stochastic_gradient_descent(
    f=lambda x, y: (x**2 + y - 7)**2 + (x - y + 1)**2,
    lr=0.015,
    x_t=[0, 0],
    max_iter=1000,
    tol=1e-3
)

(3.83185466235675e-7, -2.999855012457979, -1.9992471632382347)